In [1]:
import pandas as pd
import os
import re
from Bio import SeqIO

In [ ]:
#Blast between wild and mutated fastas --> default outfmt 6
blast_results = "blast_output.txt"

blast_cols = [
    "qseqid", "sseqid", "pident", "length", "mismatch", "gapopen",
    "qstart", "qend", "sstart", "send", "evalue", "bitscore"
]

blast_df = pd.read_csv(blast_results, sep="\t", names=blast_cols)
blast_df

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,Wild_contig_1,Mutated_contig_2,100.000,583708,0,0,1,583708,583708,1,0.000000e+00,1078000.0
1,Wild_contig_1,Mutated_contig_2,94.501,1582,80,7,1907,3485,578428,576851,0.000000e+00,2433.0
2,Wild_contig_1,Mutated_contig_2,94.501,1582,80,7,5281,6858,581802,580224,0.000000e+00,2433.0
3,Wild_contig_1,Mutated_contig_2,92.908,282,3,7,178659,178923,404861,404580,5.470000e-107,394.0
4,Wild_contig_1,Mutated_contig_2,92.908,282,3,7,178848,179129,405050,404786,5.470000e-107,394.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4053,Wild_contig_115,Mutated_contig_4,98.462,195,3,0,1,195,378290,378484,1.670000e-95,344.0
4054,Wild_contig_115,Mutated_contig_4,94.972,179,9,0,1,179,378386,378564,1.320000e-76,281.0
4055,Wild_contig_115,Mutated_contig_4,94.040,151,9,0,45,195,378238,378388,4.870000e-61,230.0
4056,Wild_contig_115,Mutated_contig_4,89.157,83,9,0,1,83,378482,378564,3.080000e-23,104.0


In [ ]:
contig_len = {}
for s in SeqIO.parse("wild_genome.fasta", "fasta"):
    contig_len[s.id] = len(s.seq)

In [6]:
# Para cada par de contigs, recuperar o alinhamento com maior pident e, em caso de empate, maior length
best_hits = blast_df.sort_values(["qseqid", "sseqid", "pident", "length"], ascending=[True, True, False, False]) 
best_hits = best_hits.groupby(["qseqid", "sseqid"], as_index=False).first()

final_list = []
for index, row in best_hits.iterrows():
    contig = row["qseqid"]
    aln_len = row["length"]
    if aln_len >= contig_len[contig] * 0.9:
        final_list.append(row)

final_best_hit = pd.DataFrame(final_list)
final_best_hit

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
12,Wild_contig_1,Mutated_contig_2,100.000,583708,0,0,1,583708,583708,1,0.000000e+00,1078000.0
33,Wild_contig_10,Mutated_contig_11,99.989,164980,16,2,1,164979,164979,1,0.000000e+00,304600.0
47,Wild_contig_100,Mutated_contig_42,100.000,265,0,0,1,265,2957,2693,2.830000e-139,490.0
49,Wild_contig_101,Mutated_contig_105,99.612,258,1,0,1,258,258,1,9.970000e-134,472.0
50,Wild_contig_101,Mutated_contig_9,99.612,258,1,0,1,258,419,162,9.970000e-134,472.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1115,Wild_contig_98,Mutated_contig_10,98.195,277,4,1,3,279,14755,15030,5.020000e-137,483.0
1122,Wild_contig_98,Mutated_contig_73,100.000,279,0,0,1,279,429,151,4.950000e-147,516.0
1123,Wild_contig_98,Mutated_contig_9,92.115,279,20,2,1,279,117058,116782,8.700000e-110,392.0
1124,Wild_contig_99,Mutated_contig_101,100.000,266,0,0,1,266,1,266,7.910000e-140,492.0


In [ ]:
import pysam
import pandas as pd

#Use the sam file of the concatenated read mapping

# Processar arquivo SAM
samfile = pysam.AlignmentFile("mapping_output.sam", "r")

# Dicionários para armazenar as posições dos reads em cada genoma
genoma1 = {}
genoma2 = {}

for read in samfile:
    if not read.is_unmapped:
        ref_name = samfile.get_reference_name(read.reference_id)
        start = read.reference_start
        end = read.reference_end
        read_name = read.query_name
        # Determinar a qual genoma pertence
        if "Wild" in ref_name:
            genoma1[read_name] = (ref_name, start, end)
        elif "Mutated" in ref_name:
            genoma2[read_name] = (ref_name, start, end)

# Encontrar reads presentes em ambos os genomas
reads_comuns = set(genoma1.keys()) & set(genoma2.keys())

# Criar DataFrame de equivalência
equivalencia = []
for read_name in reads_comuns:
    ref1, start1, end1 = genoma1[read_name]
    ref2, start2, end2 = genoma2[read_name]
    equivalencia.append({
        "read_name": read_name,
        "contig_name_wild": ref1,
        "read_start_wild": start1,
        "read_end_wild": end1,
        "contig_name_mutated": ref2,
        "read_start_mutated": start2,
        "read_end_mutated": end2
    })

df_equivalencia = pd.DataFrame(equivalencia)
df_equivalencia

,read_name,contig_name_wild,read_start_wild,read_end_wild,contig_name_mutated,read_start_mutated,read_end_mutated
0,MNG005:30:AAGWH3HM5:1:1202:50365:47440,Wild_contig_15,68274,68525,Mutated_contig_15,45194,45445
1,MNG005:30:AAGWH3HM5:1:2114:61934:1757,Wild_contig_21,8119,8370,Mutated_contig_18,8119,8370
2,MNG005:30:AAGWH3HM5:1:1404:62048:43483,Wild_contig_3,409799,410039,Mutated_contig_3,106209,106449
3,MNG005:30:AAGWH3HM5:1:1405:14880:21882,Wild_contig_15,29039,29290,Mutated_contig_15,84429,84680
4,MNG005:30:AAGWH3HM5:1:1113:34686:36800,Wild_contig_4,180705,180956,Mutated_contig_4,180704,180955
...,...,...,...,...,...,...,...
601627,MNG005:30:AAGWH3HM5:1:2301:26828:17622,Wild_contig_1,170796,171047,Mutated_contig_2,412661,412912
601628,MNG005:30:AAGWH3HM5:1:2307:13403:38769,Wild_contig_13,31312,31563,Mutated_contig_1,637112,637363
601629,MNG005:30:AAGWH3HM5:1:2411:45271:19553,Wild_contig_1,199159,199410,Mutated_contig_2,384298,384549
601630,MNG005:30:AAGWH3HM5:1:2413:10828:27940,Wild_contig_9,94061,94312,Mutated_contig_10,94061,94312


In [7]:
# df_equivalencia.to_csv("Equivalent_read_mapping.csv", index=False)

In [8]:
# Integrar DataFrame de equivalência dos reads com resultados do BLAST (formato 6)
import pandas as pd

blast_df = final_best_hit.copy()

# Função para checar se um alinhamento de read está dentro de uma região BLAST
# (considera que qseqid é o contig wild, sseqid é o contig mutated)
def is_equivalent(row, blast_df):
    # Filtra BLAST para pares de contigs correspondentes
    hits = blast_df[(blast_df["qseqid"] == row["contig_name_wild"]) &
                    (blast_df["sseqid"] == row["contig_name_mutated"])]
    for _, hit in hits.iterrows():
        # Checa se a posição do read está dentro da região alinhada
        if (row["read_start_wild"] >= min(hit["qstart"], hit["qend"]) and
            row["read_end_wild"] <= max(hit["qstart"], hit["qend"]) and
            row["read_start_mutated"] >= min(hit["sstart"], hit["send"]) and
            row["read_end_mutated"] <= max(hit["sstart"], hit["send"])):
            return True
    return False

# Aplica a função ao DataFrame de equivalência
df_equivalencia["equivalente_BLAST"] = df_equivalencia.apply(lambda row: is_equivalent(row, blast_df), axis=1)

df_equivalencia.head()

,read_name,contig_name_wild,read_start_wild,read_end_wild,contig_name_mutated,read_start_mutated,read_end_mutated,equivalente_BLAST
0,MNG005:30:AAGWH3HM5:1:1202:50365:47440,Wild_contig_15,68274,68525,Mutated_contig_15,45194,45445,True
1,MNG005:30:AAGWH3HM5:1:2114:61934:1757,Wild_contig_21,8119,8370,Mutated_contig_18,8119,8370,True
2,MNG005:30:AAGWH3HM5:1:1404:62048:43483,Wild_contig_3,409799,410039,Mutated_contig_3,106209,106449,True
3,MNG005:30:AAGWH3HM5:1:1405:14880:21882,Wild_contig_15,29039,29290,Mutated_contig_15,84429,84680,True
4,MNG005:30:AAGWH3HM5:1:1113:34686:36800,Wild_contig_4,180705,180956,Mutated_contig_4,180704,180955,True


In [9]:
final_df = df_equivalencia[df_equivalencia["equivalente_BLAST"]]

In [10]:
final_df

,read_name,contig_name_wild,read_start_wild,read_end_wild,contig_name_mutated,read_start_mutated,read_end_mutated,equivalente_BLAST
0,MNG005:30:AAGWH3HM5:1:1202:50365:47440,Wild_contig_15,68274,68525,Mutated_contig_15,45194,45445,True
1,MNG005:30:AAGWH3HM5:1:2114:61934:1757,Wild_contig_21,8119,8370,Mutated_contig_18,8119,8370,True
2,MNG005:30:AAGWH3HM5:1:1404:62048:43483,Wild_contig_3,409799,410039,Mutated_contig_3,106209,106449,True
3,MNG005:30:AAGWH3HM5:1:1405:14880:21882,Wild_contig_15,29039,29290,Mutated_contig_15,84429,84680,True
4,MNG005:30:AAGWH3HM5:1:1113:34686:36800,Wild_contig_4,180705,180956,Mutated_contig_4,180704,180955,True
...,...,...,...,...,...,...,...,...
601625,MNG005:30:AAGWH3HM5:1:2102:28968:41798,Wild_contig_40,3246,3497,Mutated_contig_20,35556,35807,True
601626,MNG005:30:AAGWH3HM5:1:2213:52239:49390,Wild_contig_3,18922,19173,Mutated_contig_3,497075,497326,True
601627,MNG005:30:AAGWH3HM5:1:2301:26828:17622,Wild_contig_1,170796,171047,Mutated_contig_2,412661,412912,True
601629,MNG005:30:AAGWH3HM5:1:2411:45271:19553,Wild_contig_1,199159,199410,Mutated_contig_2,384298,384549,True


In [11]:
final_df.to_csv("Equivalent_read_mapping.csv", index=False)